pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

url = "https://www.fda.gov/science-research/clinical-trials-and-human-subject-protection/clinical-trials-guidance-documents"
driver.get(url)

tables = driver.find_elements("tag name", "table")

table_html = tables[0].get_attribute("outerHTML")

fda_df = pd.read_html(table_html)

driver.quit()

C:\Users\ganga.babu\AppData\Local\Temp\ipykernel_21488\2448604318.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fda_df = pd.read_html(table_html)


In [11]:
type(fda_df[0])

pandas.core.frame.DataFrame

In [13]:
fda_df[0].head()

,Guidance Title,Topic,Draft or Final,Date Issued
0,Processes and Practices Applicable to Bioresea...,"Clinical Trials, Administrative / Procedural",Draft,6/05/2024
1,Cancer Clinical Trial Eligibility Criteria: Pe...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024
2,Cancer Clinical Trial Eligibility Criteria: Wa...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024
3,Cancer Clinical Trial Eligibility Criteria: La...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024
4,Informed Consent,Good Clinical Practice (GCP),Final,8/15/2023


In [ ]:
fda_df[0].iloc[0]

Guidance Title    Processes and Practices Applicable to Bioresea...
Topic                  Clinical Trials, Administrative / Procedural
Draft or Final                                                Draft
Date Issued                                               6/05/2024
Name: 0, dtype: object

In [16]:
fda_df[0].iloc[0,0]

'Processes and Practices Applicable to Bioresearch Monitoring Inspections'

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from io import StringIO

options = Options()
# Remove headless to debug if needed; add if confident
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

try:
    # Step 1: Load main guidance documents page
    url = "https://www.fda.gov/science-research/clinical-trials-and-human-subject-protection/clinical-trials-guidance-documents"
    driver.get(url)

    # Step 2: Wait until the first table element appears (max 20 secs)
    table = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    table_html = table.get_attribute("outerHTML")

    # Parse HTML table — wrap in StringIO to avoid warning
    main_df = pd.read_html(StringIO(table_html))[0]

    # Extract guidance links from the first table column anchor tags
    guidance_links = []
    title_cells = driver.find_elements(By.CSS_SELECTOR, "table tbody tr td:first-child a")
    for a in title_cells:
        guidance_links.append(a.get_attribute("href"))

    main_df['Guidance Link'] = guidance_links

    # Step 3: For each guidance link, fetch additional info
    rows = []
    for link in guidance_links:
        driver.get(link)
        time.sleep(2)  # wait for page content to load

        # Extract PDF download link (if any)
        pdf_link = None
        try:
            pdf_button = driver.find_element(By.CSS_SELECTOR, "a.btn-primary[href$='.pdf'], a.btn-primary[href*='/download']")
            pdf_link = pdf_button.get_attribute("href")
        except:
            pdf_link = None

        # Extract Docket Number
        docket_number = None
        try:
            dt_elements = driver.find_elements(By.CSS_SELECTOR, "dl.lcds-description-list--grid dt")
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.lcds-description-list--grid dd")
            for dt, dd in zip(dt_elements, dd_elements):
                if "Docket Number" in dt.text:
                    docket_number = dd.text.strip()
                    break
        except:
            docket_number = None

        # Extract Issuing Offices (list)
        issuing_offices = []
        try:
            office_divs = driver.find_elements(By.CSS_SELECTOR, ".field--name-field-issuing-office-taxonomy .field--item")
            issuing_offices = [office.text for office in office_divs if office.text.strip()]
        except:
            issuing_offices = []

        # Extract Description Paragraph (first <p> within main content)
        description = ""
        try:
            p_tag = driver.find_element(By.CSS_SELECTOR, "main p")
            description = p_tag.text.strip()
        except:
            description = ""

        rows.append({
            "Guidance URL": link,
            "PDF Link": pdf_link,
            "Docket Number": docket_number,
            "Issuing Offices": issuing_offices,
            "Description": description,
        })

finally:
    driver.quit()

# Convert collected detailed data to a DataFrame
details_df = pd.DataFrame(rows)

# Merge main data table and detailed info on guidance URLs
final_df = main_df.merge(details_df, left_on='Guidance Link', right_on='Guidance URL', how='left')

# # Preview the combined data
# print(final_df.head())

# # Save to CSV file
# final_df.to_csv("fda_clinical_guidance_details.csv", index=False)


In [22]:
# # Save to CSV file
final_df.to_csv("fda_clinical_guidance_details.csv", index=False)

In [43]:
final_df.to_excel("Clinical_Guidance_Downloads.xlsx", index=False)

In [ ]:
final_df.iloc[0]

Guidance Title     Processes and Practices Applicable to Bioresea...
Topic                   Clinical Trials, Administrative / Procedural
Draft or Final                                                 Draft
Date Issued                                                6/05/2024
Guidance Link      https://www.fda.gov/regulatory-information/sea...
Guidance URL       https://www.fda.gov/regulatory-information/sea...
PDF Link                   https://www.fda.gov/media/179027/download
Docket Number                                        FDA-2023-D-5021
Issuing Offices    [Office of Inspections and Investigations, Off...
Description                                        GUIDANCE DOCUMENT
Name: 0, dtype: object

In [26]:
final_df.iloc[0,3]

'6/05/2024'

In [25]:
final_df.iloc[0,4]

'https://www.fda.gov/regulatory-information/search-fda-guidance-documents/processes-and-practices-applicable-bioresearch-monitoring-inspections'

In [27]:
final_df.iloc[0,5]

'https://www.fda.gov/regulatory-information/search-fda-guidance-documents/processes-and-practices-applicable-bioresearch-monitoring-inspections'

In [28]:
final_df.iloc[0,6]

'https://www.fda.gov/media/179027/download'

In [32]:
final_df.iloc[0,0]

'Processes and Practices Applicable to Bioresearch Monitoring Inspections'

In [33]:
print(final_df.iloc[0,0])
print(final_df.iloc[0,6])

Processes and Practices Applicable to Bioresearch Monitoring Inspections
https://www.fda.gov/media/179027/download


In [ ]:
final_df.iloc[i,0]
final_df.iloc[i,6]

In [31]:
final_df.iloc[1,6]

'https://www.fda.gov/media/178018/download'

In [21]:
final_df.head()

,Guidance Title,Topic,Draft or Final,Date Issued,Guidance Link,Guidance URL,PDF Link,Docket Number,Issuing Offices,Description
0,Processes and Practices Applicable to Bioresea...,"Clinical Trials, Administrative / Procedural",Draft,6/05/2024,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/media/179027/download,FDA-2023-D-5021,"[Office of Inspections and Investigations, Off...",GUIDANCE DOCUMENT
1,Cancer Clinical Trial Eligibility Criteria: Pe...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/media/178018/download,FDA-2024-D-1377,"[Oncology Center of Excellence, Center for Dru...",GUIDANCE DOCUMENT
2,Cancer Clinical Trial Eligibility Criteria: Wa...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/media/178016/download,FDA-2024-D-1376,"[Oncology Center of Excellence, Center for Dru...",GUIDANCE DOCUMENT
3,Cancer Clinical Trial Eligibility Criteria: La...,"Clinical Trials, Clinical - Medical",Draft,4/25/2024,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/media/178013/download,FDA-2024-D-1402,"[Oncology Center of Excellence, Center for Dru...",GUIDANCE DOCUMENT
4,Informed Consent,Good Clinical Practice (GCP),Final,8/15/2023,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/regulatory-information/sea...,https://www.fda.gov/media/88915/download,FDA-2006-D-0031,"[Office of the Commissioner, Office of Clinica...",GUIDANCE DOCUMENT


In [42]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Guidance Title   139 non-null    object
 1   Topic            139 non-null    object
 2   Draft or Final   139 non-null    object
 3   Date Issued      139 non-null    object
 4   Guidance Link    139 non-null    object
 5   Guidance URL     139 non-null    object
 6   PDF Link         119 non-null    object
 7   Docket Number    107 non-null    object
 8   Issuing Offices  139 non-null    object
 9   Description      139 non-null    object
dtypes: object(10)
memory usage: 11.0+ KB


10% works

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Directory for saving PDFs
save_path = r"D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf"
os.makedirs(save_path, exist_ok=True)

options = Options()
# Set headless if desired
options.add_argument("--headless")
# Set Chrome preferences for PDF auto-download
prefs = {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,  # Avoid opening pdf in-browser
    "download.default_directory": save_path
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

try:
    for i in range(final_df.shape[0]):
        file_name = final_df.iloc[i, 0]
        pdf_url = final_df.iloc[i, 6]
        if pd.isna(pdf_url):
            continue

        # Clean file name
        safe_name = "".join(c if c.isalnum() or c in " ._-" else "_" for c in file_name)
        output_path = os.path.join(save_path, f"{safe_name}.pdf")

        # Check if file already exists
        if os.path.isfile(output_path):
            print(f"{output_path} already exists, skipping.")
            continue

        driver.get(pdf_url)
        print(f"Downloading {pdf_url} to {output_path}")

        # Wait for the download to finish
        # This method is simple—consider a more robust method for large scale
        time.sleep(5)

except Exception as e:
    print("Error occurred:", e)
finally:
    driver.quit()

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Directory for saving PDFs
save_path = r"D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf"
os.makedirs(save_path, exist_ok=True)

options = Options()
# Set headless if desired
options.add_argument("--headless")
# Set Chrome preferences for PDF auto-download
prefs = {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,  # Avoid opening pdf in-browser
    "download.default_directory": save_path
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

try:
    for i in range(final_df.shape[0]):
        file_name = final_df.iloc[i, 0]
        pdf_url = final_df.iloc[i, 6]
        if pd.isna(pdf_url):
            continue

        # Clean file name
        safe_name = "".join(c if c.isalnum() or c in " ._-" else "_" for c in file_name)
        output_path = os.path.join(save_path, f"{safe_name}.pdf")

        # Check if file already exists
        if os.path.isfile(output_path):
            print(f"{output_path} already exists, skipping.")
            continue

        driver.get(pdf_url)
        print(f"Downloading {pdf_url} to {output_path}")

        # Wait for the download to finish
        # This method is simple—consider a more robust method for large scale
        time.sleep(5)

except Exception as e:
    print("Error occurred:", e)
finally:
    driver.quit()

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Load your DataFrame (ensure final_df is defined)
# final_df = pd.read_csv('your_csv_file.csv')  # Or however you're loading it

# Directory to save PDFs
save_path = r"D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf"
os.makedirs(save_path, exist_ok=True)

# Log file for failed downloads
log_file_path = os.path.join(save_path, "failed_downloads.log")

# Setup Chrome options
options = Options()
options.add_argument("--headless")
prefs = {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,
    "download.default_directory": save_path
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

def wait_for_download(download_dir, timeout=30):
    """
    Polls the download directory until there are no .crdownload files left,
    indicating downloads are complete.
    """
    seconds = 0
    while seconds < timeout:
        if not any(fname.endswith(".crdownload") for fname in os.listdir(download_dir)):
            return True
        time.sleep(1)
        seconds += 1
    return False

try:
    for i in range(final_df.shape[0]):
        file_name = final_df.iloc[i, 0]
        pdf_url = final_df.iloc[i, 6]

        if pd.isna(pdf_url):
            continue

        # Clean filename
        safe_name = "".join(c if c.isalnum() or c in " ._-" else "_" for c in file_name)
        output_path = os.path.join(save_path, f"{safe_name}.pdf")

        if os.path.isfile(output_path):
            print(f"{output_path} already exists, skipping.")
            continue

        # Track current files before downloading
        before_files = set(os.listdir(save_path))

        # Navigate to PDF URL to trigger download
        driver.get(pdf_url)
        print(f"Started downloading: {pdf_url}")

        if wait_for_download(save_path):
            # Detect the new file
            after_files = set(os.listdir(save_path))
            new_files = after_files - before_files
            downloaded_file = None
            for f in new_files:
                if f.lower().endswith(".pdf"):
                    downloaded_file = os.path.join(save_path, f)
                    break

            if downloaded_file:
                os.rename(downloaded_file, output_path)
                print(f"Downloaded and saved as: {output_path}")
            else:
                raise Exception("PDF not found after download")

        else:
            raise TimeoutError("Download timed out")

except Exception as e:
    with open(log_file_path, "a", encoding="utf-8") as log_file:
        log_file.write(f"{file_name} - {str(e)}\n")
    print(f"Failed to download: {file_name} -> {e}")

finally:
    driver.quit()
    print("Process completed.")


Started downloading: https://www.fda.gov/media/179027/download
Failed to download: Processes and Practices Applicable to Bioresearch Monitoring Inspections -> PDF not found after download
Process completed.


In [49]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Load DataFrame (replace with your loading logic)
# final_df = pd.read_csv('your_data.csv')

# Target download directory
download_dir = r"D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf"
os.makedirs(download_dir, exist_ok=True)

# Log file path
log_file_path = os.path.join(download_dir, "failed_downloads.log")

# Configure Chrome options
options = Options()
options.add_argument("--headless=new")  # Use new headless mode (more stable)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

def wait_for_pdf_download(folder, timeout=30):
    """
    Waits until a .crdownload file is replaced by a .pdf in the given folder.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        files = os.listdir(folder)
        downloading = [f for f in files if f.endswith(".crdownload")]
        finished = [f for f in files if f.endswith(".pdf")]
        if not downloading and finished:
            return finished[0]  # Return the first finished PDF
        time.sleep(1)
    return None

try:
    for i in range(final_df.shape[0]):
        file_name = final_df.iloc[i, 0]
        pdf_url = final_df.iloc[i, 6]

        if pd.isna(pdf_url):
            continue

        # Clean filename
        safe_name = "".join(c if c.isalnum() or c in " ._-" else "_" for c in file_name)
        final_file_path = os.path.join(download_dir, f"{safe_name}.pdf")

        if os.path.exists(final_file_path):
            print(f"Already exists: {final_file_path}")
            continue

        # Track files before download
        before = set(os.listdir(download_dir))

        print(f"Starting download: {pdf_url}")
        driver.get(pdf_url)

        # Wait for the download to finish
        downloaded_filename = wait_for_pdf_download(download_dir, timeout=60)

        if downloaded_filename:
            downloaded_path = os.path.join(download_dir, downloaded_filename)
            os.rename(downloaded_path, final_file_path)
            print(f"Download complete: {final_file_path}")
        else:
            raise Exception("PDF download failed or timed out")

except Exception as e:
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(f"{file_name} - {str(e)}\n")
    print(f"Failed to download: {file_name} -> {e}")

finally:
    driver.quit()
    print("All downloads processed.")


Starting download: https://www.fda.gov/media/179027/download
Download complete: D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf\Processes and Practices Applicable to Bioresearch Monitoring Inspections.pdf
Starting download: https://www.fda.gov/media/178018/download
Download complete: D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf\Cancer Clinical Trial Eligibility Criteria_ Performance Status.pdf
Starting download: https://www.fda.gov/media/178016/download
Download complete: D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf\Cancer Clinical Trial Eligibility Criteria_ Washout Periods and Concomitant Medications.pdf
Starting download: https://www.fda.gov/media/178013/download
Download complete: D:\Docketry\IPFS_deploy\OLD\PFAEBACH\Multi AGENTS\ClinicalGuidance_Pdf\Cancer Clinical Trial Eligibility Criteria_ Laboratory Values.pdf
Starting download: https://www.fda.gov/media/88915/download
Download complete: D:\Docketry\IPF